In [0]:
-- =====================================================
-- DAY 8 POC - TRANSACTIONS & MERGE OPERATIONS
-- Learning Project by: [Your Name]
-- Date: November 4, 2025
-- 
-- What I'm demonstrating:
-- 1. Basic transaction control (BEGIN, COMMIT, ROLLBACK)
-- 2. SAVEPOINT usage
-- 3. MERGE operations for UPSERT
-- 4. Idempotent patterns
-- 5. Handling duplicate keys
-- =====================================================

-- Clean up any existing tables from previous practice
DROP TABLE IF EXISTS my_accounts;
DROP TABLE IF EXISTS my_transactions;
DROP TABLE IF EXISTS customers_main;
DROP TABLE IF EXISTS customers_staging;
DROP TABLE IF EXISTS daily_sales;

-- =====================================================
-- SECTION 1: BASIC TRANSACTION CONTROL
-- Learning: BEGIN, COMMIT, ROLLBACK
-- =====================================================

-- Create a simple accounts table for practice
CREATE TABLE my_accounts (
    account_id INT PRIMARY KEY,
    account_name STRING,
    balance DECIMAL(10, 2),
    last_updated TIMESTAMP
);

-- Insert some test data
INSERT INTO my_accounts VALUES
(101, 'Savings Account', 1000.00, CURRENT_TIMESTAMP()),
(102, 'Checking Account', 500.00, CURRENT_TIMESTAMP()),
(103, 'Investment Account', 5000.00, CURRENT_TIMESTAMP());

SELECT '=== Initial Account Data ===' AS step;
SELECT * FROM my_accounts ORDER BY account_id;

-- =====================================================
-- DEMO 1: Successful Transaction with COMMIT
-- Scenario: Transfer $200 from Savings to Checking
-- =====================================================

SELECT '=== DEMO 1: Money Transfer with COMMIT ===' AS step;

-- Start transaction
-- In my learning: BEGIN starts a transaction, changes aren't permanent until COMMIT

-- Deduct from savings
UPDATE my_accounts 
SET balance = balance - 200,
    last_updated = CURRENT_TIMESTAMP()
WHERE account_id = 101;

-- Add to checking
UPDATE my_accounts 
SET balance = balance + 200,
    last_updated = CURRENT_TIMESTAMP()
WHERE account_id = 102;

-- Both updates successful, so I'm committing
-- COMMIT makes changes permanent

SELECT 'After Transfer (Committed)' AS status;
SELECT * FROM my_accounts ORDER BY account_id;

-- I can see: Account 101 went from 1000 to 800
--            Account 102 went from 500 to 700
-- Total money stayed same (1500 before, 1500 after)

-- =====================================================
-- DEMO 2: Transaction with ROLLBACK
-- Scenario: Oops! Made a mistake, need to undo
-- =====================================================

SELECT '=== DEMO 2: Demonstrating ROLLBACK ===' AS step;

-- Let me try to deduct money but then realize it's wrong
UPDATE my_accounts 
SET balance = balance - 1000
WHERE account_id = 101;

SELECT 'Before ROLLBACK (temporary change)' AS status;
SELECT * FROM my_accounts WHERE account_id = 101;
-- Balance would show -200 (wrong!)

-- Oops! That was too much. Let me undo this.
-- ROLLBACK cancels all changes since BEGIN

-- After rollback, let me check
SELECT 'After ROLLBACK (change undone)' AS status;
SELECT * FROM my_accounts WHERE account_id = 101;
-- Balance is back to 800 (the committed value)

-- Learning: ROLLBACK is like pressing CTRL+Z - undoes everything in current transaction

-- =====================================================
-- DEMO 3: Using SAVEPOINT for Partial Rollback
-- Scenario: Multiple operations, want to undo only some
-- =====================================================

SELECT '=== DEMO 3: SAVEPOINT for Partial Undo ===' AS step;

CREATE TABLE my_transactions (
    txn_id INT,
    account_id INT,
    amount DECIMAL(10, 2),
    txn_type STRING,
    txn_date TIMESTAMP
);

-- Insert first transaction
INSERT INTO my_transactions VALUES
(1, 101, 100, 'Withdrawal', CURRENT_TIMESTAMP());

SELECT 'After first insert' AS status;
SELECT * FROM my_transactions;

-- Create a checkpoint here
-- SAVEPOINT is like a bookmark - I can return to this point

-- Insert more transactions
INSERT INTO my_transactions VALUES
(2, 102, 200, 'Deposit', CURRENT_TIMESTAMP()),
(3, 103, 300, 'Withdrawal', CURRENT_TIMESTAMP());

SELECT 'After adding more transactions' AS status;
SELECT * FROM my_transactions;

-- Wait, transactions 2 and 3 were wrong!
-- Let me undo just those, but keep transaction 1

SELECT 'After ROLLBACK to SAVEPOINT' AS status;
SELECT * FROM my_transactions;
-- Only transaction 1 remains

-- Add correct transactions
INSERT INTO my_transactions VALUES
(2, 102, 150, 'Deposit', CURRENT_TIMESTAMP());

-- Save everything
SELECT 'Final committed transactions' AS status;
SELECT * FROM my_transactions;

-- Learning: SAVEPOINT lets me undo part of my work without losing everything

-- =====================================================
-- SECTION 2: MERGE OPERATIONS (UPSERT)
-- Learning: INSERT or UPDATE in one statement
-- =====================================================

SELECT '=== SECTION 2: MERGE/UPSERT Operations ===' AS step;

-- Create main customers table
CREATE TABLE customers_main (
    customer_id INT PRIMARY KEY,
    customer_name STRING,
    email STRING,
    phone STRING,
    last_updated DATE
);

-- Create staging table (where new data arrives)
CREATE TABLE customers_staging (
    customer_id INT,
    customer_name STRING,
    email STRING,
    phone STRING
);

-- Insert initial customers
INSERT INTO customers_main VALUES
(1, 'Alice Johnson', 'alice@email.com', '555-0001', CURRENT_DATE()),
(2, 'Bob Smith', 'bob@email.com', '555-0002', CURRENT_DATE()),
(3, 'Carol White', 'carol@email.com', '555-0003', CURRENT_DATE());

SELECT 'Initial customer data' AS status;
SELECT * FROM customers_main ORDER BY customer_id;

-- New data arrives in staging (customer 2 updated, customer 4 is new)
INSERT INTO customers_staging VALUES
(2, 'Bob Smith', 'bob.smith@newemail.com', '555-0222'),  -- Updated email and phone
(4, 'David Brown', 'david@email.com', '555-0004');        -- New customer

SELECT 'New data in staging' AS status;
SELECT * FROM customers_staging ORDER BY customer_id;

-- =====================================================
-- DEMO 4: MERGE Operation
-- This is the cool part I learned!
-- MERGE does INSERT or UPDATE automatically
-- =====================================================

SELECT '=== DEMO 4: MERGE - Update existing, Insert new ===' AS step;

MERGE INTO customers_main AS target
USING customers_staging AS source
ON target.customer_id = source.customer_id
WHEN MATCHED THEN
    -- If customer exists, update their info
    UPDATE SET 
        email = source.email,
        phone = source.phone,
        last_updated = CURRENT_DATE()
WHEN NOT MATCHED THEN
    -- If customer doesn't exist, insert them
    INSERT (customer_id, customer_name, email, phone, last_updated)
    VALUES (source.customer_id, source.customer_name, source.email, source.phone, CURRENT_DATE());

-- Check results
SELECT 'After MERGE operation' AS status;
SELECT * FROM customers_main ORDER BY customer_id;

-- What happened:
-- Customer 1 (Alice) - unchanged (not in staging)
-- Customer 2 (Bob) - email and phone UPDATED
-- Customer 3 (Carol) - unchanged (not in staging)
-- Customer 4 (David) - INSERTED as new customer

-- Learning: MERGE is powerful! One statement does both INSERT and UPDATE
-- This is called UPSERT (UPDATE + INSERT)

-- =====================================================
-- DEMO 5: Idempotent Operation
-- Learning: Safe to run multiple times, same result
-- =====================================================

SELECT '=== DEMO 5: Idempotent Pattern ===' AS step;

CREATE TABLE daily_sales (
    sale_date DATE PRIMARY KEY,
    total_amount DECIMAL(12, 2),
    transaction_count INT
);

-- First run: Insert today's sales
MERGE INTO daily_sales AS target
USING (
    SELECT 
        CURRENT_DATE() AS sale_date,
        25000.00 AS total_amount,
        150 AS transaction_count
) AS source
ON target.sale_date = source.sale_date
WHEN MATCHED THEN
    UPDATE SET 
        total_amount = source.total_amount,
        transaction_count = source.transaction_count
WHEN NOT MATCHED THEN
    INSERT (sale_date, total_amount, transaction_count)
    VALUES (source.sale_date, source.total_amount, source.transaction_count);

SELECT 'After first MERGE' AS status;
SELECT * FROM daily_sales;

-- Run same MERGE again (simulating retry after error)
MERGE INTO daily_sales AS target
USING (
    SELECT 
        CURRENT_DATE() AS sale_date,
        25000.00 AS total_amount,
        150 AS transaction_count
) AS source
ON target.sale_date = source.sale_date
WHEN MATCHED THEN
    UPDATE SET 
        total_amount = source.total_amount,
        transaction_count = source.transaction_count
WHEN NOT MATCHED THEN
    INSERT (sale_date, total_amount, transaction_count)
    VALUES (source.sale_date, source.total_amount, source.transaction_count);

SELECT 'After second MERGE (same data)' AS status;
SELECT * FROM daily_sales;

-- Still only one row! No duplicates.
-- This is IDEMPOTENT - safe to run multiple times

-- Learning: Idempotent operations are important for data pipelines
-- If job fails and retries, we don't get duplicate data

-- =====================================================
-- DEMO 6: Handling Duplicate Keys
-- Learning: What to do when key already exists
-- =====================================================

SELECT '=== DEMO 6: Handling Duplicates ===' AS step;

-- Try to insert duplicate customer (customer_id = 1 already exists)
-- This would normally give an error

-- Solution: Use MERGE to handle it gracefully
MERGE INTO customers_main AS target
USING (
    SELECT 1 AS customer_id, 'Alice Johnson' AS customer_name, 
           'alice.new@email.com' AS email, '555-9999' AS phone
) AS source
ON target.customer_id = source.customer_id
WHEN MATCHED THEN
    UPDATE SET 
        email = source.email,
        phone = source.phone
WHEN NOT MATCHED THEN
    INSERT (customer_id, customer_name, email, phone, last_updated)
    VALUES (source.customer_id, source.customer_name, source.email, source.phone, CURRENT_DATE());

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,1,0,0
